# Homework 3

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [18]:
import math
import numpy as np
from scipy.stats import wrapcauchy, levy_stable
import param
import panel as pn
import plotly.graph_objects as go

pn.extension('plotly')

# CLASSES

In [19]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

# FUNCTIONS

In [20]:
def brownian_motion_3d(num_steps, speed, start_pos=(0, 0, 0)):
    """
    Function to simulate 3D Brownian motion
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
    Returns:
        trajectory: 3D trajectory of the particle
    """
    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position

    velocity = Vec2d(speed, 0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1): # simulate the motion

        # generate a random angle to turn the velocity vector
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
        
        # update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x, velocity.y, velocity.get_length()])
    
    return trajectory

def correlated_random_walk_3d(num_steps, speed, start_pos=(0, 0, 0), cauchy_coefficient=0.9):
    """
    Function to simulate 3D correlated random walk
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
        cauchy_coefficient: Coefficient of the Cauchy distribution
    Returns:
        trajectory: 3D trajectory of the particle
    """
    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position

    velocity = Vec2d(speed, 0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1): # simulate the motion

        # generate a random angle to turn the velocity vector
        angle = wrapcauchy.rvs(cauchy_coefficient)
        velocity = velocity.rotated(angle)

        # update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x, velocity.y, velocity.get_length()])
    
    return trajectory


def levy_flight_3d(num_steps, speed, start_pos=(0, 0, 0), cauchy_coefficient=0.7, alpha=1.5):
    """
    Function to simulate 3D Levy flight
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
        cauchy_coefficient: Coefficient of the Cauchy distribution
        alpha: Alpha parameter of the Levy
    Returns:
        trajectory: 3D trajectory of the particle
    """

    beta = 0 # Beta parameter of the Levy
    time_per_step = 1.5 # Time to reach the next point

    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position


    velocity = Vec2d(speed,0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1):
        step_length = levy_stable.rvs(alpha,beta) # Generate a random step length

        # Generate a random angle to turn the velocity vector
        angle = wrapcauchy.rvs(cauchy_coefficient)
        velocity = velocity.rotated(angle)

        # Distance / velocity = time
        time = step_length / velocity.get_length() * time_per_step # Time to reach the next point
        
        # Update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x,velocity.y, time])
    
    return trajectory



def calculate_path_length(trajectory):
    """
    Function to calculate the path length of a trajectory
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        path_length: Path length of the trajectory
    """

    # Calculate the distance between consecutive points
    distances = np.sqrt(np.sum(np.diff(trajectory, axis=0)**2, axis=1))
    # Calculate the path length
    path_length = np.cumsum(distances)
    # Calculate the steps
    steps = np.arange(len(path_length))
    return steps, path_length

def calculate_msd(trajectory):
    """
    Function to calculate the mean squared displacement of a trajectory
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        msd: Mean squared displacement of the trajectory
    """

    steps = np.arange(len(trajectory)) # Steps of the trajectory
    start_pos = trajectory[0] # Initial position of the particle

    # Calculate the squared distances
    squared_distances = np.sum((trajectory - start_pos)**2, axis=1)
    return steps, squared_distances

def calculate_turning_angle_distribution(trajectory):
    """
    Function to calculate the turning angle distribution of
    Arguments:
        trajectory: 3D trajectory of the particle
    Returns:
        steps: Steps of the trajectory
        angles: Turning angles of the trajectory
    """
    if len(trajectory) < 3:
        return np.array([]), np.array([])
    
    # Calculate the directions of the trajectory
    directions = np.diff(trajectory, axis=0)
    
    # Calculate the angles between consecutive directions
    angles = []
    for i in range(len(directions)-1):
        v1 = directions[i]
        v2 = directions[i+1]
        
        # Normalize the vectors
        v1_norm = v1 / np.linalg.norm(v1) if np.linalg.norm(v1) > 0 else v1
        v2_norm = v2 / np.linalg.norm(v2) if np.linalg.norm(v2) > 0 else v2
        
        # Calculate the dot product
        dot_product = np.dot(v1_norm, v2_norm)
        # Clip the dot product to avoid numerical errors
        dot_product = np.clip(dot_product, -1.0, 1.0)
        
        # Calculate the angle
        angle = np.arccos(dot_product)
        angles.append(angle)
    
    steps = np.arange(len(angles))
    return steps, np.array(angles)



## Class TrajectoryDashboard

In [21]:

class TrajectoryDashboard(param.Parameterized):
    rw_type = param.ObjectSelector(default="BM", objects=["BM", "CRW", "LF"])
    num_steps = param.Integer(default=500, bounds=(10, 5000))
    speed = param.Number(default=1.0, bounds=(0.1, 10.0))
    start_pos_x = param.Number(default=0.0)
    start_pos_y = param.Number(default=0.0)
    start_pos_z = param.Number(default=0.0)
    cauchy_coefficient = param.Number(default=0.7, bounds=(0.0, 1.0))
    levy_exponent = param.Number(default=1.5, bounds=(1.0, 3.0))
    metric_type = param.ObjectSelector(default="MSD", objects=["PL", "MSD", "TAD"])
    
    def __init__(self, **params):
        super(TrajectoryDashboard, self).__init__(**params)
        self._current_trajectory = None
        
        # Create the Plotly panes for the trajectory and metric plots
        self.trajectory_pane = pn.pane.Plotly(height=400)
        self.metric_pane = pn.pane.Plotly(height=400)
        
        # Generate the initial trajectory and update the plots
        self.update_plots()
        
        # Watch the parameters to update the plots
        self.param.watch(self.update_plots, ['rw_type', 'num_steps', 'speed', 
                                           'start_pos_x', 'start_pos_y', 'start_pos_z',
                                           'cauchy_coefficient', 'levy_exponent', 'metric_type'])
    
    def generate_trajectory(self):
        """
        Function to generate a 3D trajectory based on the selected random walk type
        Returns:
            trajectory: 3D trajectory of the particle
        """

        # Assign the start position based on the parameters
        start_pos = (self.start_pos_x, self.start_pos_y, self.start_pos_z)
        
        if self.rw_type == "BM":
            trajectory = brownian_motion_3d(self.num_steps, self.speed, start_pos)
        elif self.rw_type == "CRW":
            trajectory = correlated_random_walk_3d(self.num_steps, self.speed, 
                                                 start_pos, self.cauchy_coefficient)
        else:
            trajectory = levy_flight_3d(self.num_steps, self.speed, start_pos, 
                                       self.cauchy_coefficient, self.levy_exponent)
        
        self._current_trajectory = trajectory
        return trajectory
    
    def create_trajectory_plot(self, trajectory):
        """
        Function to create a Plotly 3D plot of the trajectory
        Arguments:
            trajectory: 3D trajectory of the particle
        Returns:
            fig: Plotly figure object
        """

        # Create a Plotly 3D scatter plot of the trajectory
        colors = {"BM": "blue", "CRW": "red", "LF": "green"}

        # Titles for the different random walk types
        titles = {
            "BM": "Brownian Motion trajectory in 3D",
            "CRW": "Correlated Random Walk trajectory in 3D",
            "LF": "Lévy Flight trajectory in 3D"
        }
        
        # Create the Plotly figure
        fig = go.Figure(data=[go.Scatter3d(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            z=trajectory[:, 2],
            mode='lines',
            line=dict(color=colors[self.rw_type], width=2),
            name=self.rw_type
        )])

        # Update the layout of the figure
        fig.update_layout(
            title=titles[self.rw_type],
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                aspectmode='cube'
            ),
            margin=dict(l=0, r=0, b=0, t=30),
            height=400
        )
        
        return fig
    
    def create_metric_plot(self, trajectory):
        """
        Function to create a Plotly plot of the selected metric
        Arguments:
            trajectory: 3D trajectory of the particle
        Returns:
            fig: Plotly figure object
        """
        if self.metric_type == "PL":
            x, y = calculate_path_length(trajectory)
            title = "Path Length"
            y_label = "PL"
            color = 'blue'
        elif self.metric_type == "MSD":
            x, y = calculate_msd(trajectory)
            title = "Mean Squared Displacement"
            y_label = "MSD"
            color = 'purple'
        else:  # TAD
            x, y = calculate_turning_angle_distribution(trajectory)
            title = "Turning Angle Distribution"
            y_label = "Angle (radians)"
            color = 'orange'
        
        # Create the Plotly figure
        fig = go.Figure(data=[go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(color=color, width=2),
            name=self.metric_type
        )])
        
        fig.update_layout(
            title=title,
            xaxis_title='Steps',
            yaxis_title=y_label,
            margin=dict(l=0, r=0, b=0, t=30),
            height=400
        )
        
        return fig
    
    def update_plots(self, *events):
        """
        Function to update the trajectory and metric plots
        Arguments:
            events: Event data
        """
        trajectory = self.generate_trajectory()
        
        # Update the Plotly panes with the new plots
        self.trajectory_pane.object = self.create_trajectory_plot(trajectory)
        self.metric_pane.object = self.create_metric_plot(trajectory)
    

    def panel(self):
        """
            This is where the fix is implemented - we create a panel with parameters that will
            be shown or hidden based on the current rw_type
        Returns:
            main_layout: Panel layout
        """

        # Basic parameters always visible
        basic_params = pn.Column(
            pn.pane.Markdown("## Basic Parameters"),
            pn.Param(self.param.rw_type, name="RW Type"),
            pn.Param(self.param.num_steps, name="Number of steps"),
            pn.Param(self.param.speed, name="Speed"),
            pn.Param(self.param.start_pos_x, name="Starting pos_x"),
            pn.Param(self.param.start_pos_y, name="Starting pos_y"),
            pn.Param(self.param.start_pos_z, name="Starting pos_z"),
        )
        
        # Create Panel widgets for the advanced parameters
        cauchy_widget = pn.Param(
            self.param.cauchy_coefficient, 
            name="Cauchy coefficient",
            # This is the key: the widget will only be visible when rw_type is CRW or LF
            visible=self.rw_type in ["CRW", "LF"]
        )
        
        levy_widget = pn.Param(
            self.param.levy_exponent, 
            name="Lévy exponent (alpha)",
            # This widget will only be visible when rw_type is LF
            visible=self.rw_type == "LF",
        )
        
        # Add watchers to update visibility when rw_type changes
        def update_cauchy_visibility(event):
            cauchy_widget.visible = event.new in ["CRW", "LF"]
            
        def update_levy_visibility(event):
            levy_widget.visible = event.new == "LF"
            
        self.param.watch(update_cauchy_visibility, 'rw_type')
        self.param.watch(update_levy_visibility, 'rw_type')
        
        # Advanced parameters column
        advanced_params = pn.Column(
            pn.pane.Markdown("## Advanced Parameters"),
            cauchy_widget,
            levy_widget
        )
        
        # Parámetros para la métrica
        metric_params = pn.Column(
            pn.pane.Markdown("## Metric Type"),
            pn.Param(self.param.metric_type, name="Metric")
        )
        
        # Layout de la interfaz
        trajectory_column = pn.Column(
            pn.pane.Markdown("## 3D Trajectory"),
            self.trajectory_pane,
            sizing_mode='stretch_width'
        )
        
        metric_column = pn.Column(
            pn.pane.Markdown("## Metrics"),
            self.metric_pane,
            sizing_mode='stretch_width'
        )
        
        # Panel de parámetros
        param_column = pn.Column(
            basic_params,
            advanced_params,
            metric_params,
            width=300
        )
        
        # Main layout with more space between the configuration panel and the graphs
        main_layout = pn.Row(
            param_column,
            pn.Spacer(width=30),
            pn.Column(
                trajectory_column,
                metric_column,
                sizing_mode='stretch_both'
            ),
            sizing_mode='stretch_both'
        )
        
        return main_layout

In [23]:
# Create the dashboard
dashboard = TrajectoryDashboard()

# Serve the dashboard using Panel
dashboard.panel().show()

Launching server at http://localhost:56471
